Importing useful libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
#print(os.getcwd())


Loading data from file

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nn_train.csv')
data.head()

ID  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
0   1         81        109         91         96        107         97   
1   2        168        168        169        169        170        171   
2   3         14         20         32          6         27          6   
3   4         60        109        157        126         82         80   
4   5         63         72         86         75         93         77   

   Feature_7  Feature_8  Feature_9  ...  Feature_1017  Feature_1018  \
0        103         95        100  ...           114            93   
1        172        172        175  ...            44            44   
2          5         34         15  ...            63            64   
3         70         58         37  ...           105           102   
4         75         67         62  ...           108           109   

   Feature_1019  Feature_1020  Feature_1021  Feature_1022  Feature_1023  \
0            91            98            72            92            78   
1            44            44            44            44            44   
2            63            60            54            48            43   
3            99            98           100           105           111   
4           102           106            96            91           102   

   Feature_1024  binary_label  class_label  
0            78             0            5  
1            44             0            9  
2            41             0            5  
3           114             1            8  
4            89             0            5  

[5 rows x 1027 columns]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Extracting and normalising the data

In [5]:
x = data.iloc[:,1:1025]
print(x.head())
x = (x - x.mean())/x.std()
print(x.head())
x = x.values
print(x.shape)
y = data.iloc[:,1026].values
print(y.shape)
print(type(y))

   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
0         81        109         91         96        107         97   
1        168        168        169        169        170        171   
2         14         20         32          6         27          6   
3         60        109        157        126         82         80   
4         63         72         86         75         93         77   

   Feature_7  Feature_8  Feature_9  Feature_10  ...  Feature_1015  \
0        103         95        100          99  ...           109   
1        172        172        175         175  ...            44   
2          5         34         15          28  ...            62   
3         70         58         37          61  ...           122   
4         75         67         62          65  ...           120   

   Feature_1016  Feature_1017  Feature_1018  Feature_1019  Feature_1020  \
0            95           114            93            91            98   
1       

Defining one hot encoding functions

In [6]:

# .values se ek column wala numpy array nhi bnta
def one_hot_encode(arr):


    unique_values = np.unique(arr) # gibr a list of unique elements in arr


    value_to_index = {val: idx for idx, val in enumerate(unique_values)} #  mapping of unique values to indices


    one_hot_matrix = np.zeros((len(arr), len(unique_values)), dtype = np.int64)        # Initialize a zero matrix of shape (len(arr), len(unique_values))


    for i, val in enumerate(arr):
        one_hot_matrix[i, value_to_index[val]] = 1    # Setting the appropriate indices to 1

    return one_hot_matrix


spilliting data for cross validation and taking transpose

In [7]:


print((y.shape))
m = x.shape[0]

n = int(0.2*m)
x_test = x[:n,:].T
x_train = x[n:,:].T
print(x_train.shape)

y_test = y[:n].T
y_train_not_one = y[n:]
print(y_test.shape)
print(y_test)

(80000,)
(1024, 64000)
(16000,)
[5 9 5 ... 9 9 7]


one hot encoding of training output

In [8]:
print(y_train_not_one)
a = np.unique(y_train_not_one)
print(a)
print(y_train_not_one.shape)
class_counts = pd.Series(y).value_counts(normalize = True)  # normalize = true krne pr proprtion deta hai
print(class_counts)
y_train = one_hot_encode(y_train_not_one).T
print(y_train.shape)
print(y_train)
print(y_train.shape)

print(y_test)


[ 6  4  6 ...  2 10  1]
[ 1  2  3  4  5  6  7  8  9 10]
(64000,)
7     0.100900
1     0.100400
9     0.100288
4     0.100175
5     0.100112
2     0.099937
3     0.099913
6     0.099850
10    0.099513
8     0.098913
Name: proportion, dtype: float64
(10, 64000)
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
(10, 64000)
[5 9 5 ... 9 9 7]


Defining functions

In [9]:
def cost_function(a3,y,l,w1,w2,w3):

    e = 1e-10  # for safety
    m = y.shape[1]
    cost  = -(1/m)*(np.sum(y*np.log(a3 + e)))+ (l/(2*m))*(np.sum(np.square(w1))+ np.sum(np.square(w2)) + np.sum(np.square(w3)))
    return cost

In [10]:

n_x = x_train.shape[0]
n_h1 = 1500
n_h2 = 1200
n_y = y_train.shape[0]
alpha = 0.01
iteration = 100
l = 0.0001

w1 = np.random.randn(n_h1,n_x)*np.sqrt(2/n_x)

b1 = np.zeros((n_h1, 1))

w2 = np.random.randn(n_h2,n_h1)*np.sqrt(2/n_h1)

b2 = np.zeros((n_h2,1))

w3 = np.random.randn(n_y,n_h2)*0.01

b3 = np.zeros((n_y,1))

def relu(x_train):
    return np.maximum(x_train, 0)
def softmax(x_train):
    expX = np.exp(x_train)
    return expX/np.sum(expX, axis=0)


def d_relu(x_train):
    return np.array(x_train > 0, dtype = np.float32)

In [ ]:
def compute_accuracy(a3,y):
    a3_index = np.argmax(a3, axis = 0) + 1
    accuracy = (np.mean(a3_index == y))*100
    return accuracy

# Initialize Adam parameters
beta1 = 0.9  # Exponential decay for the first moment estimates
beta2 = 0.999  # Exponential decay for the second moment estimates
epsilon = 1e-8  # Small constant to avoid division by zero
alpha_initial = 0.001  # Learning rate
decay_rate = 0.96
decay_step = 100
l = 0.01
# Initialize moment estimates for all parameters
m_w1, m_w2, m_w3 = 0, 0, 0  # First moments for weights
v_w1, v_w2, v_w3 = 0, 0, 0  # Second moments for weights
m_b1, m_b2, m_b3 = 0, 0, 0  # First moments for biases
v_b1, v_b2, v_b3 = 0, 0, 0  # Second moments for biases

t = 0  # Time step for bias correction

j_history = []
for i in range(iteration):
    # Forward propagation
    z1 = np.dot(w1, x_train) + b1
    a1 = relu(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = relu(z2)
    z3 = np.dot(w3, a2) + b3
    a3 = softmax(z3)

    # Backward propagation
    m = x.shape[1]
    dz3 = (a3 - y_train)
    dw3 = (1/m) * (np.dot(dz3, a2.T) + l * np.sum(w3))
    db3 = (1/m) * np.sum(dz3, axis=1, keepdims=True)
    dz2 = (1/m) * np.dot(w3.T, dz3) * d_relu(a2)
    dw2 = (1/m) * (np.dot(dz2, a1.T) + l * np.sum(w2))
    db2 = (1/m) * np.sum(dz2, axis=1, keepdims=True)
    dz1 = (1/m) * np.dot(w2.T, dz2) * d_relu(a1)
    dw1 = (1/m) * (np.dot(dz1, x_train.T) + l * np.sum(w1))
    db1 = (1/m) * np.sum(dz1, axis=1, keepdims=True)

    # Update Adam parameters
    t += 1
    alpha = alpha_initial * (decay_rate ** (t // decay_step))
    # Update weights and biases using Adam
    for param, grad, m, v in zip(
        [w1, w2, w3, b1, b2, b3],
        [dw1, dw2, dw3, db1, db2, db3],
        [m_w1, m_w2, m_w3, m_b1, m_b2, m_b3],
        [v_w1, v_w2, v_w3, v_b1, v_b2, v_b3],
    ):
        m = beta1 * m + (1 - beta1) * grad  # Update biased first moment
        v = beta2 * v + (1 - beta2) * (grad ** 2)  # Update biased second moment
        m_corrected = m / (1 - beta1 ** t)  # Correct bias for first moment
        v_corrected = v / (1 - beta2 ** t)  # Correct bias for second moment
        param -= alpha * m_corrected / (np.sqrt(v_corrected) + epsilon)  # Update parameter

    # Compute and store cost
    if i < iteration:
        c = cost_function(a3, y_train, l, w1, w2, w3)
        j_history.append(c)

    # Print cost at intervals
    if (i % (iteration // 10)) == 0:
      accuracy = compute_accuracy(a3, y_train_not_one)
      print(f"Iteration {i}, Cost: {c}",Accuracy: {accuracy})

# Plot cost history
plt.plot(j_history)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.title("Cost Reduction Over Iterations")
plt.show()


ValueError: shapes (1500,1024) and (80000,1024) not aligned: 1024 (dim 1) != 80000 (dim 0)

In [12]:
def prediction(x_test,w1,w2,w3,b1,b2,b3):
    z1 = np.dot(w1,x_test) + b1
    a1 = relu(z1)
    z2 = np.dot(w2,a1) + b2
    a2 = relu(z2)
    z3 = np.dot(w3,a2) + b3
    a3 = softmax(z3)


    print((a3.shape))

    a3_index = np.argmax(a3, axis = 0) + 1

    print(a3_index)
    return a3_index
a3_index = prediction(x_test,w1,w2,w3,b1,b2,b3)


class_counts = pd.Series(a3_index).value_counts(normalize = True)  # normalize = true krne pr proprtion deta hai
print(class_counts)
accuracy = np.mean(a3_index == y_test)
print(f"accuracy of cross validation set is {accuracy*100}")

a3_train_index = prediction(x_train,w1,w2,w3,b1,b2,b3)
class_counts = pd.Series(a3_train_index).value_counts(normalize = True)  # normalize = true krne pr proprtion deta hai
print(class_counts)
print(a3_train_index)
print(y_train_not_one)
print(y_train_not_one.shape)
print(f"accuracy of training set is {np.mean(a3_train_index == y_train_not_one)*100}")

(10, 16000)
[7 9 5 ... 9 9 7]
7     0.116625
10    0.106625
1     0.103750
9     0.101688
2     0.100437
8     0.096437
4     0.094312
3     0.094250
6     0.093500
5     0.092375
Name: proportion, dtype: float64
accuracy of cross validation set is 56.16875
(10, 64000)
[ 3  4  6 ...  2 10  6]
7     0.111203
10    0.108516
9     0.103563
1     0.101766
2     0.099953
8     0.099391
6     0.097172
4     0.094000
3     0.092594
5     0.091844
Name: proportion, dtype: float64
[ 3  4  6 ...  2 10  6]
[ 6  4  6 ...  2 10  1]
(64000,)
accuracy of training set is 70.09531249999999


In [13]:

def f1_score_avg(y,y_test,i,f1_score):
    p = 0
    r = 0

    if i > len(f1_score):
        return np.mean(f1_score)
    else:
        p = np.sum((y == i) & (y_test == i))/(np.sum((y == i)))# & (y_test != i)) + np.sum((y == i) & (y_test == i)))
        r = np.sum((y == i) & (y_test == i))/(np.sum((y_test == i)))# + np.sum((y == i) & (y_test == i)))
        f1_score[i-1] = 2*(p*r)/(p + r)

        return f1_score_avg(y,y_test,i+1,f1_score)
f1_score = np.zeros((n_y))
i = 1
f1_avg = f1_score_avg(a3_index,y_test,i ,f1_score)
print(f1_avg)



0.5603288902120683
